## 1. Imports & Env Set-up

In [1]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.WARNING)

In [2]:
import os
from IPython.display import Markdown, display

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

In [3]:
from haystack import Pipeline
from haystack import Document
from haystack.dataclasses import ChatMessage

from haystack.utils.auth import Secret
from haystack.components.builders import PromptBuilder

from haystack.components.preprocessors.document_splitter import DocumentSplitter
from haystack.components.writers import DocumentWriter

from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

from haystack.components.generators import OpenAIGenerator
from haystack.components.embedders import OpenAIDocumentEmbedder
from haystack.components.embedders import OpenAITextEmbedder


## 2. Indexing Pipeline

In [4]:
document_store = InMemoryDocumentStore()

In [5]:
import os
import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import BytesIO

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_STORAGE_CONNECTION_STRING"))

# Get a client for the specific container
container_client = blob_service_client.get_container_client("prod-container")

# Specify the exact path of the folder
genie_product='brand_pulse'
client_name='kenvue'
product_category='US_skincare'
folder_path = f"outputs/{genie_product}/{client_name}/{product_category}/bot_output/"

# List blobs in the exact folder
blob_list = container_client.list_blobs(name_starts_with=folder_path)
# Initialize a list to store documents
documents = []
dfs = []

for blob in blob_list:
    blob_client = container_client.get_blob_client(blob)
    blob_extension = blob.name.split('.')[-1].lower()

    # Download and read the blob content based on file type
    if blob_extension == 'csv':
        blob_content = blob_client.download_blob().readall()
        df = pd.read_csv(BytesIO(blob_content))
    elif blob_extension == 'parquet':
        blob_content = blob_client.download_blob().readall()
        df = pd.read_parquet(BytesIO(blob_content))
    else:
        continue  # Skip if it's not a CSV or Parquet file

    # Convert the entire DataFrame to a JSON string
    documents.append({
        'content': df.to_json(orient='records'),  # Converting the DataFrame to JSON with records orientation
        'name': blob.name
    })

    dfs.append(df)

In [6]:
# Convert your documents into Haystack Documents and write them to the store
haystack_documents = [Document(content=doc['content'], meta={"name": doc['name']}) for doc in documents]
len(haystack_documents)

1

In [7]:
splitter = DocumentSplitter(split_length = 200, split_overlap = 10, split_threshold = 20)
embedder = OpenAIDocumentEmbedder(model="text-embedding-3-small")
writer = DocumentWriter(document_store=document_store)

indexing = Pipeline()
indexing.add_component("splitter", splitter)
indexing.add_component("embedder", embedder)
indexing.add_component("writer", writer)

indexing.connect("splitter", "embedder")
indexing.connect("embedder", "writer")

🚅 Components
  - splitter: DocumentSplitter
  - embedder: OpenAIDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - splitter.documents -> embedder.documents (List[Document])
  - embedder.documents -> writer.documents (List[Document])

In [8]:
indexing.run({ "splitter": {"documents": haystack_documents } })

Calculating embeddings:  84%|████████▎ | 51/61 [01:00<00:13,  1.31s/it]

In [9]:
import yaml
import os
import json

file_path = "field_description.yaml"
yaml_file_path = os.path.join(os.getcwd(), file_path)

with open(file_path, "r") as file:
    field_data = yaml.safe_load(file)

df_name = "summary"
table_description_dict = field_data.get(f"{df_name}").get("table_description", " ")
field_descriptions_dict = field_data.get(f"{df_name}").get("field_description", " ")

table_description = json.dumps(table_description_dict)
field_descriptions = json.dumps(field_descriptions_dict)

## 3. Custom Components

In [10]:
system_prompt = """You are an 25+ year experienced `Advanced Business Analyst & counsellor` with statistical capability and domain expertise.
Your primary task is to assist non-technical users in analyzing data with deep dive along with inference and suggestions if needed.
Don't assume random data, in any case.
"""

### 3.1 RAG Components

In [11]:
map_prompt = """
Answer Strictly only for the asked original question in format of Enterprise Grade Business Case Study ( dont't mention).
Include focused analysis for all possible nuances in the asked question.
If the answer is not contained within the context, reply with clarification questions. Don't assume random data, in any case.

```
Context:
{% for doc in documents %}
   {{ doc.content }}
{% endfor %}
```

```
Table descriptions:
{{table_description}}

Fields/Columns description:
{{field_descriptions}}
```
Instructions:
1. Respond insighfully in crisp, structured and bulleted formats with facts, numbers (strictly up to 2 decimal only) and inferences.
2. Strictly Avoid mentioning phrases like "Based on the shared context/dataset" etc.
3. When need to provide analysis, Use quantitative values to provide factual accuracy instead of relying upon the text mentions.

Question: {{ query }}
"""

reduce_prompt = """Given the conversation history and the provided supporting documents, give a brief answer to the question.
Note that supporting documents are not part of the conversation. If question can't be answered from supporting documents, say so & if needed ask clarification questions.

Answer Strictly only for the asked original question in format of Enterprise Grade Business Case Study ( dont't mention).
Include focused analysis for all possible nuances in the asked question.
If the answer is not contained within the context, reply with clarification questions. Don't assume random data.

    Conversation history:
    {% for memory in memories %}
        {{ memory.content }}
    {% endfor %}

    Supporting documents:
    {% for doc in documents %}
        {{ doc }}
    {% endfor %}

Instructions:
1. Respond insighfully as a "Pro Business Analyst" in well-structured and bulleted formats with facts, numbers (strictly up to 2 decimal only) and drawn inferences (TLdr).
2. Strictly Avoid mentioning phrases like "Based on the shared context/dataset", "mentioned", "shared" etc.
3. When need to provide analysis, Use quantitative values to provide factual accuracy instead of relying upon the text mentions.

\nQuestion: {{query}}
\nAnswer:
"""

In [12]:
from haystack import component, Document
import concurrent.futures
from typing import List

@component
class BatchGenerator:
    outgoing_edges = 1

    def __init__(self, question_prompt, system_prompt, table_description, field_descriptions, batch_size: int = 10):
        self.question_prompt = question_prompt
        self.system_prompt = system_prompt
        self.batch_size = batch_size
        self.table_description=table_description
        self.field_descriptions=field_descriptions

    @component.output_types(replies=List)
    def run(self, documents: List[Document], user_asked_query: str, **kwargs):

        self.user_asked_query = user_asked_query
        batches_of_10 = [documents[i:i + self.batch_size] for i in range(0, len(documents), self.batch_size)]
        print(f"Total Batch Size {len(batches_of_10)} from {len(documents)} documents")
        # Process batches in parallel
        with concurrent.futures.ThreadPoolExecutor() as executor:
            batch_results = list(executor.map(self.process_batch, batches_of_10))

        # Combine all batch results into one final output
        # final_result = "\n".join(batch_results)
        # print(final_result)
        return {"replies": batch_results}

    def process_batch(self, batch_doc: List[Document]):

        prompt_builder = PromptBuilder(template=self.question_prompt)
        generator = OpenAIGenerator(model="gpt-4o-mini"
                                    , system_prompt=self.system_prompt
                                    , generation_kwargs={
                                        'temperature':0,
                                        'seed':101,
                                        'n':1
                                    })

        batch_rag = Pipeline()
        batch_rag.add_component("prompt", prompt_builder)
        batch_rag.add_component("generator", generator)
        batch_rag.connect("prompt", "generator")
        # Generate output for individual batch
        response = batch_rag.run({"prompt": {"documents": batch_doc
                                             , "query": self.user_asked_query
                                             , "table_description":self.table_description
                                             , "field_descriptions":self.field_descriptions
                                }})

        return response["generator"]["replies"][0]

### 3.2 DATAFRAME CONTEXT Component

In [13]:
from haystack import component
from pandasai.llm import OpenAI
from pandasai import Agent
from response_parser import GenieResponse
from pandasai.connectors import PandasConnector

import json
import yaml
import os
from typing import List
import pandas as pd

@component
class DataContextGenerator:

    def __init__(self):
        self.llm_model = "gpt-4o-mini"
        self.temperature = 0.2
        self.memory_size = 25

    def load_yaml(self, file_path):
        """Load YAML file synchronously."""
        with open(file_path, "r") as file:
            return yaml.safe_load(file)

    def run_chat(self, prompt, agent):
        """Run the agent and handle DataFrame conversion to JSON."""
        try:
            result = agent.chat(prompt, output_type='dataframe')

            if isinstance(result, str):
                return result
            else:
                return ""
        except Exception as e:
            return ""

    def run_agent(self, question, df):
        # Load the YAML file (assumes it's in the current directory)
        file_path = "field_description.yaml"
        yaml_file_path = os.path.join(os.getcwd(), file_path)
        field_data = self.load_yaml(yaml_file_path)

        df_name = "summary"

        connector = PandasConnector({"original_df": df}
                            , name=f"{df_name}"
                            , description=field_data.get(f"{df_name}").get("table_description", " ")
                            , field_descriptions=field_data.get(f"{df_name}").get("field_description", " "))

        # Initialize LLM
        llm = OpenAI(model=self.llm_model, temperature=self.temperature, seed=10)

        # Create the agent
        agent = Agent(
            connector,
            memory_size=self.memory_size,
            description="""You are a 25+ year experienced `Pro Python & Pandas Assistant`. You specialize in developing precise dataframe generation code.
                Your primary task is to assist non-technical users by providing relevant dataframe to start the analysis, by aggregating at all relevant granularity.
                If the answer is not contained within the documents, reply with 'no_data'.
                By default consider Latest `Year_Month` data, unless asked specifically otherwise.

                ```
                Instructions:
                1. STRICTLY AVOID CHARTS, REVERT ONLY DATAFRAME.
                2. ENSURE TO DO SANITY CHECKS & HANDLE ERRORS INTERNALLY TO ANSWER.
                3. WHEN AGGREGATING AT YEARS OR MONTHS, CONSIDER ONLY THAT PART AS STRING AND CHANGE COLUMN NAME ACCORDINGLY.
                4. WHEN FETCHING HIGHEST, STRICTLY CONSIDER ALL MATCHING ITEMS WITH SIMILAR VALUES TO INCLUDE IN RESPONSE.
                ```
                """,
            config={
                "llm": llm,
                "open_charts": False,
                "save_charts": False,
                "verbose": False,
                "save_logs": False,
                "response_parser": GenieResponse,
                "max_retries": 2
            }
        )

        # Rephrase query and ask clarification questions
        rephrased_prompt = agent.rephrase_query(question)
        # clarification_questions = agent.clarification_questions(str(question))

        # Run the initial prompt and rephrased prompt concurrently
        response_dict = dict()

        response_dict[question], response_dict[rephrased_prompt] = self.run_chat(question, agent), self.run_chat(rephrased_prompt, agent)

        # Run clarification questions concurrently
        # clarifications_responses = [self.run_chat(que, agent) for que in clarification_questions]

        # for que, resp in zip(clarification_questions, clarifications_responses):
        #     response_dict[que] = resp

        return response_dict

    @component.output_types(replies=str)
    def run(self, user_asked_query: str, df_list: List, **kwargs):
        data_context_dict = self.run_agent(user_asked_query, df_list[0])
        return {"replies": json.dumps(data_context_dict)}


In [14]:
import tiktoken

@component
class ContextWindowLimiter:

    def trim_context(self, paragraph: str, max_tokens=100000):
        tokenizer = tiktoken.get_encoding("cl100k_base")  # Assuming GPT-4 tokenizer
        tokens = tokenizer.encode(paragraph)

        if len(tokens) <= max_tokens:
            return paragraph

        trimmed_tokens = tokens[-max_tokens:]
        trimmed_paragraph = tokenizer.decode(trimmed_tokens)

        return trimmed_paragraph

    def count_tokens(self, paragraph: str):
        tokenizer = tiktoken.get_encoding("cl100k_base")
        return len(tokenizer.encode(paragraph))

    @component.output_types(replies=str)
    def run(self, paragraph: str, **kwargs):
        limited_paragraph = self.trim_context(paragraph)
        return {"replies": limited_paragraph}

In [15]:

data_context_prompt = """Given the conversation history and the provided supporting documents, give a brief answer to the question.
Note that supporting documents are not part of the conversation. If question can't be answered from supporting documents, say so & if needed ask clarification questions.

Answer Strictly only for the asked original question in format of Enterprise Grade Business Case Study ( dont't mention).
Include focused analysis for all possible nuances in the asked question.
Here are the relevant dataframes extracted for the user asked questions and some clarification questions as well as context.

    Conversation history:
    {% for memory in memories %}
        {{ memory.content }}
    {% endfor %}

    Supporting documents:
    {{documents}}

Instructions:
1. Have facts, numbers (strictly up to 2 decimal only) and drawn inferences (TLdr).
2. Strictly Avoid mentioning phrases like "Based on the shared context/dataset", "mentioned", "shared" etc.
3. For numeric values, ensure to provide data in efficient PIVOT form tabular grids, if more than 2 rows, as per relevance and provide insights from it in reference to nuances of questions asked.
4. When analyse, prioritize only latest of months / years data, unless asked specifically otherwise.
5. Ensure to provide an enterprise grade answer in sections, but only limit to asked question.

\nQuestion: {{query}}
\nAnswer:
"""

### 3.3 Pipeline Selector

In [16]:
selector_prompt = """Based on the conversation history, user question, and the table structure with column descriptions, decide which ONE of the following should be used for the next step of analysis:

1. **RAG Pipeline**: Choose this if:
   - The question involves analyzing textual or semi-structured data embedded in the table (e.g., free-text fields, long descriptions, or mixed-format data).
   - The user is asking for insights that might require retrieving or embedding data across multiple sources or documents
   - The user question involves a broad analysis that isn't directly about performing calculations or operations on specific columns of the table.

2. **DATAFRAME Pipeline**: Choose this if:
   - The question directly/indirectly references numerical columns or operations
   - The user is asking for a straightforward statistical or numerical operation that can be executed directly on the dataset (e.g., sorting, filtering, grouping by a column).

3. **Generic Response**: Choose this ONLY if:
   - The user input is clearly a greeting, a generic comment, or irrelevant text that does not require any analytical pipeline.

### Decision Rules:
- **For Textual or Semi-Structured Data**: If the table contains mixed-format data or text fields (e.g., comments, descriptions), and the question involves broad or interpretive insights, choose the `RAG Pipeline`.
- **For Numerical Data**: If the question involves direct operations on numerical data (e.g., sums, averages, comparisons), choose the `DATAFRAME Pipeline`.
- **For Mixed Questions**: When both text and numerical elements are involved, select the pipeline that aligns best with the core requirement of the question (e.g., is the focus more on numeric calculations or text-based insights?).
- **Prioritize the `DATAFRAME Pipeline`**: Use the `DATAFRAME Pipeline` for any question that involves direct operations on numerical or structured data, even if the question includes some general context. Only choose the `RAG Pipeline` if the core of the question clearly requires textual analysis.
- **Limit `RAG Pipeline`** to situations where structured numerical analysis cannot solve the problem, such as when dealing with broad natural language queries or complex text-based analysis.
- **Fallback to the `DATAFRAME Pipeline`** if there is ambiguity between numerical and textual analysis.
- **Fallback to `Generic Response`** for anything irrelevant, greetings, or broad queries unrelated to the table data and including external calculations, not relevant to data.

### Table Structure and Column Descriptions:
    ```
    Table descriptions:
    {{table_description}}

    Fields/Columns description:
    {{field_descriptions}}
    ```

Carefully analyze the conversation history and the user question. Choose ONLY ONE option based on the best analysis of the input.

    Conversation history:
    {% for memory in memories %}
        {{ memory.content }}
    {% endfor %}

    User Question:
    {{query}}

Return only one: `RAG Pipeline`, `DATAFRAME Pipeline`, or `GENERIC Response`.
"""

In [17]:
routes = [
    {
        "condition": "{{'rag' in replies[0]|lower}}",
        "output": "{{query}}",
        "output_name": "go_to_rag",
        "output_type": str,
    },
    {
        "condition": "{{'dataframe' in replies[0]|lower}}",
        "output": "{{query}}",
        "output_name": "go_to_df",
        "output_type": str,
    },
    {
        "condition": "{{'generic' in replies[0]|lower}}",
        "output": "{{query}}",
        "output_name": "go_to_generic",
        "output_type": str,
    },
    {
        "condition": "{{'generic' not in replies[0]|lower}}",
        "output": "{{replies[0]}}",
        "output_name": "answer",
        "output_type": str,
    },
]

In [18]:
# from haystack import Pipeline
# from haystack.dataclasses import ChatMessage

# from haystack.components.builders import ChatPromptBuilder, PromptBuilder
# from haystack.components.generators.chat import OpenAIChatGenerator
# from haystack.components.routers import ConditionalRouter

# router = ConditionalRouter(routes=routes)


# demo_pipe = Pipeline()
# demo_pipe.add_component("selector_prompt", ChatPromptBuilder(template=[ChatMessage.from_user(selector_prompt)]))
# demo_pipe.add_component("llm", OpenAIChatGenerator(model="gpt-4o-mini", generation_kwargs={'temperature':0, 'seed':101, 'n':1 }))
# demo_pipe.add_component("router", ConditionalRouter(routes=routes))

# demo_pipe.connect("selector_prompt.prompt", "llm.messages")
# demo_pipe.connect("llm.replies", "router.replies")

# while True:
#     question = input("Enter your question or Q to exit.\n🧑 ")
#     if question=="Q":
#         break
#     print(question)
#     response = demo_pipe.run( data={
#         "selector_prompt": {"query": question, "table_description":table_description, "field_descriptions":field_descriptions},
#         "router": {"query": question},
#         })
#     print(response)
#     # print(response["llm"]["replies"][0].content)


### 3.4 Generic Component

In [19]:
generic_response_prompt = """You are Genie Bot, an advanced and friendly analytical assistant developed by i-Genie.
Given the conversation history, and mainly Based on the user's message, respond appropriately, whether it's a greeting, generic question, or analytical inquiry.
Adjust your tone according to the user's sentiment, whether positive, neutral, or negative, while always maintaining professionalism.

### Response Instructions:

1. **If the user greets you**:
   - Respond with enthusiasm: "Hello! It's great to hear from you! 😊 How can I assist you today with your data analysis?"

2. **If the user asks a generic or irrelevant question**:
   - Attempt to answer based on general knowledge, but suggest the user ask more data-related questions for better insights.
   - Politely guide the user with examples of business analysis questions, dynamically generated based on the provided data description.

3. **If the user provides positive feedback**:
   - Acknowledge their satisfaction: "I'm glad that was helpful! 😊"
   - Then offer further analysis suggestions tailored to the data: "Would you like to explore more detailed insights based on the current dataset?"

4. **If the user responds negatively**:
   - Stay calm and acknowledge potential limitations: "I understand this might not be what you expected, and I’m still learning. Sometimes, I may not be perfect."
   - Encourage collaboration: "Let’s explore further together. Based on the dataset, you can ask specific questions that may help refine the analysis."
   - Reassure them: "Your feedback helps me improve, and we can work together to find the right insights."

5. **If the user asks a data-related question**:
   - Provide the analysis as requested, and then offer additional suggestions for further exploration, based on the dataset.

6. **Always suggest dynamic follow-up questions** based on the table and data description, without revealing schema details. Use the dataset to generate relevant, meaningful questions dynamically to keep the conversation engaging.

````
### Underlying dataset Reference:
Below is the general description of the dataset, which should guide your suggestions:
    Table descriptions:
    {{table_description}}

    Fields/Columns description:
    {{field_descriptions}}
````

    Conversation history:
    {% for memory in memories %}
        {{ memory.content }}
    {% endfor %}

### Task:
1. Judge the user's message and tone to respond accordingly, whether it's a greeting, irrelevant query, positive or negative feedback, or data-related question.
2. Offer helpful business analysis question examples, crafted based on the data description. You should be dynamically tailoring responses and follow-up questions based on the data description and user input.
3. Encourage deeper data exploration, keeping your tone friendly and professional, while guiding the user toward meaningful insights.

Keep your tone friendly, professional, and energetic where appropriate! You are here to help the user make the most of the tool!

\n **User Message**: {{query}}
\n **Response**:
"""

## 4. Pipeline development

In [20]:
from typing import List
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder, PromptBuilder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators import OpenAIGenerator
from haystack.components.joiners import BranchJoiner
from haystack.components.routers import ConditionalRouter

from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore
from haystack_experimental.components.retrievers import ChatMessageRetriever
from haystack_experimental.components.writers import ChatMessageWriter

########################################
##   CUSTOM COMPONENT INSTANTIATION   ##
########################################

# Memory Components Instantiation
memory_store = InMemoryChatMessageStore()
memory_retriever = ChatMessageRetriever(memory_store)
memory_writer = ChatMessageWriter(memory_store)

# Custom Components Instantiation
batch_generator = BatchGenerator(question_prompt=map_prompt
                                 , system_prompt=system_prompt
                                 , table_description=table_description
                                 , field_descriptions=field_descriptions
                                 , batch_size=5)

data_context_generator = DataContextGenerator()

# Router Instantiation
routes = [
    {
        "condition": "{{'rag' in replies[0]|lower}}",
        "output": "{{query}}",
        "output_name": "go_to_rag",
        "output_type": str,
    },
    {
        "condition": "{{'dataframe' in replies[0]|lower}}",
        "output": "{{query}}",
        "output_name": "go_to_df",
        "output_type": str,
    },
    {
        "condition": "{{'generic' in replies[0]|lower}}",
        "output": "{{query}}",
        "output_name": "go_to_generic",
        "output_type": str,
    },
    {
        "condition": "{{'generic' not in replies[0]|lower}}",
        "output": "{{replies[0]}}",
        "output_name": "answer",
        "output_type": str,
    },
]
router = ConditionalRouter(routes=routes)

########################################
##     PIPELINE DEFINITION            ##
########################################

pipeline = Pipeline()

# components for Selected Routing
pipeline.add_component("s_prompt", ChatPromptBuilder(template=[ChatMessage.from_user(selector_prompt)]))
pipeline.add_component("s_llm", OpenAIChatGenerator(model="gpt-4o-mini", generation_kwargs={'temperature':0, 'seed':101, 'n':1 }))
pipeline.add_component("s_router", ConditionalRouter(routes=routes))

# components for Generic Response
pipeline.add_component("g_prompt", ChatPromptBuilder(template=[ChatMessage.from_user(generic_response_prompt)]))
pipeline.add_component("g_llm", OpenAIChatGenerator(model="gpt-4o-mini", generation_kwargs={'temperature':0, 'seed':101, 'n':1 }))

# components for DataFrame Context
pipeline.add_component("dc_generator", data_context_generator)
pipeline.add_component("dc_prompt_builder", ChatPromptBuilder(template=[ChatMessage.from_user(data_context_prompt)]))
pipeline.add_component("dc_llm", OpenAIChatGenerator(model="gpt-4o-mini", generation_kwargs={'temperature':0, 'seed':101, 'n':1 }))

# components for RAG
pipeline.add_component("r_query_embedder", OpenAITextEmbedder())
pipeline.add_component("r_retriever", InMemoryEmbeddingRetriever(document_store=document_store))
pipeline.add_component("r_batch_generator", batch_generator)
pipeline.add_component("r_prompt_builder", ChatPromptBuilder(template=[ChatMessage.from_user(reduce_prompt)]))
pipeline.add_component("r_llm", OpenAIChatGenerator(model="gpt-4o-mini", generation_kwargs={'temperature':0, 'seed':101, 'n':1 }))

# components for memory
pipeline.add_component("memory_retriever", memory_retriever)
pipeline.add_component("memory_writer", memory_writer)
pipeline.add_component("memory_joiner", BranchJoiner(List[ChatMessage]))

# connections for Selected routing
pipeline.connect("s_prompt.prompt", "s_llm.messages")
pipeline.connect("s_llm.replies", "s_router.replies")
pipeline.connect("s_router.go_to_rag", "r_query_embedder.text")
pipeline.connect("s_router.go_to_df", "dc_generator.user_asked_query")
pipeline.connect("s_router.go_to_generic", "g_prompt.query")

# connection for Generic Response
pipeline.connect("g_prompt.prompt", "g_llm.messages")

# connections for RAG
pipeline.connect("r_query_embedder.embedding", "r_retriever.query_embedding")
pipeline.connect("r_retriever.documents", "r_batch_generator.documents")
pipeline.connect("r_batch_generator.replies", "r_prompt_builder.documents")
pipeline.connect("r_prompt_builder.prompt", "r_llm.messages")
pipeline.connect("r_llm.replies", "memory_joiner")

# connections for DataFrame Context
pipeline.connect("dc_generator.replies", "dc_prompt_builder.documents")
pipeline.connect("dc_prompt_builder.prompt", "dc_llm.messages")
pipeline.connect("dc_llm.replies", "memory_joiner")

# connections for memory
pipeline.connect("memory_joiner", "memory_writer")
pipeline.connect("memory_retriever", "s_prompt.memories")
pipeline.connect("memory_retriever", "dc_prompt_builder.memories")
pipeline.connect("memory_retriever", "r_prompt_builder.memories")
pipeline.connect("memory_retriever", "g_prompt.memories")

🚅 Components
  - s_prompt: ChatPromptBuilder
  - s_llm: OpenAIChatGenerator
  - s_router: ConditionalRouter
  - g_prompt: ChatPromptBuilder
  - g_llm: OpenAIChatGenerator
  - dc_generator: DataContextGenerator
  - dc_prompt_builder: ChatPromptBuilder
  - dc_llm: OpenAIChatGenerator
  - r_query_embedder: OpenAITextEmbedder
  - r_retriever: InMemoryEmbeddingRetriever
  - r_batch_generator: BatchGenerator
  - r_prompt_builder: ChatPromptBuilder
  - r_llm: OpenAIChatGenerator
  - memory_retriever: ChatMessageRetriever
  - memory_writer: ChatMessageWriter
  - memory_joiner: BranchJoiner
🛤️ Connections
  - s_prompt.prompt -> s_llm.messages (List[ChatMessage])
  - s_llm.replies -> s_router.replies (List[ChatMessage])
  - s_router.go_to_rag -> r_query_embedder.text (str)
  - s_router.go_to_df -> dc_generator.user_asked_query (str)
  - s_router.go_to_generic -> g_prompt.query (str)
  - g_prompt.prompt -> g_llm.messages (List[ChatMessage])
  - dc_generator.replies -> dc_prompt_builder.documents 

In [21]:
# pipeline.show()

## 5. Conversational Bot

In [24]:
system_message = ChatMessage.from_system(system_prompt)

while True:
    messages = [system_message,]
    question = input("Enter your question or Q to exit.\n🧑 ")
    if question=="Q":
        break

    res = pipeline.run(data={"s_prompt": {"query": question, "table_description":table_description, "field_descriptions":field_descriptions},
                             "g_prompt": {"table_description":table_description, "field_descriptions":field_descriptions},
                             "s_router": {"query": question},
                             "dc_generator": {"df_list":[df,]},
                             "dc_prompt_builder": {"query": question},
                             "r_retriever": {"top_k": 50},
                             "r_batch_generator": {"user_asked_query": question,},
                             "r_prompt_builder": {"query": question},
                             "memory_joiner": {"value": [ChatMessage.from_user(question)]}
                             },
                            include_outputs_from=['g_llm', 'dc_llm', 'r_llm', "s_prompt", "s_router",
                                                  "dc_generator", "dc_prompt_builder",
                                                  "r_retriever", "r_batch_generator", "r_prompt_builder" ]
                        )

    llm_type = [key for key in ['g_llm', 'dc_llm', 'r_llm'] if key in res][0]
    assistant_resp = res[llm_type]['replies'][0]
    display(Markdown(f"{assistant_resp.content}"))

Hello! It's great to hear from you! 😊 How can I assist you today with your data analysis? If you have any specific questions or areas you'd like to explore, feel free to share!

I appreciate you asking! My day has been great, thank you! 😊 How can I assist you today with your data analysis? If you have any specific questions or topics in mind, feel free to share! For example, you might want to explore brand performance metrics or customer sentiment trends. Let’s dive in!

I understand that it can be overwhelming at times! Let's explore some options together. If you're looking for insights from your data, here are a few questions you might consider:

1. **How does the brand's Pulse Score vary across different channels?**
2. **What trends can we identify in the Affinity score over the past few months?**
3. **Which product categories are showing the highest levels of Advocacy among consumers?**
4. **How does the Trust score compare between your brand and competitors?**

Feel free to ask about any specific area you're interested in, and we can dive deeper into the analysis! 😊

### Brand Pulse Score Analysis Across Channels

#### Overview
The brand's Pulse Score provides insights into customer sentiment and engagement across various channels. This analysis focuses on the average Pulse Score, standard deviation, and the range of scores to understand the brand's performance.

#### Pulse Score Summary

| Channel | Average Pulse Score | Standard Deviation | Minimum Pulse Score | Maximum Pulse Score | Count |
|---------|---------------------|--------------------|---------------------|---------------------|-------|
| Mixed   | 57.99               | 9.77               | 20.40               | 74.04               | 920   |

#### Insights
- **Average Pulse Score**: The average Pulse Score across the mixed channels is 57.99, indicating a moderately positive sentiment.
- **Variability**: The standard deviation of 9.77 suggests a moderate level of variability in customer sentiment, indicating that while many customers feel positively, there are notable outliers.
- **Range of Scores**: The minimum score of 20.40 and maximum score of 74.04 highlight a significant range in customer experiences, suggesting that while some customers have a very negative perception, others have a highly positive view.
- **Sample Size**: With a count of 920, the data is robust, providing a reliable basis for analysis.

#### Conclusion
The brand's Pulse Score across channels shows a generally positive sentiment with considerable variability. This indicates opportunities for improvement in customer experience, particularly in addressing the lower scores to enhance overall brand perception.

### Affinity Score Trends Analysis

#### Overview
The Affinity Score reflects customer loyalty and emotional connection to the brand. Analyzing the trends over the past few months provides insights into customer sentiment and brand performance.

#### Affinity Score Summary

| Year_Month       | Average Affinity Score |
|-------------------|------------------------|
| 2023_06 - June    | 72.87                  |
| 2023_07 - July    | 72.71                  |
| 2023_08 - August   | 72.82                  |
| 2023_09 - September | 72.81                  |
| 2023_10 - October  | 71.66                  |
| 2023_11 - November  | 71.56                  |
| 2023_12 - December  | 71.48                  |

#### Insights
- **Recent Performance**: The Affinity Score peaked in June 2023 at 72.87, indicating strong customer loyalty during that month.
- **Declining Trend**: Following June, there is a noticeable decline in the Affinity Score, dropping to 71.66 in October and further to 71.48 by December. This suggests a potential decrease in customer engagement or satisfaction.
- **Stability in Scores**: The scores from July to September show relative stability, fluctuating slightly around the 72.7 mark, indicating consistent customer sentiment during that period.
- **End of Year Drop**: The significant drop in October and subsequent months may warrant further investigation into customer feedback or market conditions that could have influenced this decline.

#### Conclusion
The Affinity Score trends indicate a peak in mid-2023 followed by a downward trajectory towards the end of the year. This trend highlights the need for strategic initiatives to understand and address the factors contributing to the decline in customer affinity.

...

Total Batch Size 10 from 50 documents


### Brand Performance Overview

#### Brands Owned by You:
1. **Neutrogena**
   - **Review Average Score**: 75.33
   - **Total Conversations**: 3,218
   - **Rating NPS**: 77.81

2. **Aveeno**
   - **Rating NPS**: 88.64
   - **Total Conversations**: 1,438

3. **Lubriderm**
   - **Review Average Score**: 66.65
   - **Total Conversations**: 82
   - **Rating NPS**: 76.64

4. **Exuviance**
   - **Review Average Score**: 73.00
   - **Total Conversations**: 119

5. **Neostrata**
   - **Review Average Score**: 71.93
   - **Total Conversations**: 80

### Comparative Analysis
- **Top Performer**: 
  - **Aveeno** leads with an NPS of **88.64**, indicating high customer loyalty and satisfaction.
  
- **Strong Engagement**: 
  - **Neutrogena** has the highest total conversations (**3,218**) among your brands, suggesting strong consumer engagement.

- **Moderate Performers**: 
  - **Exuviance** and **Neostrata** have review average scores of **73.00** and **71.93**, respectively, indicating moderate consumer satisfaction but lower engagement with only **119** and **80** conversations.

- **Underperformer**: 
  - **Lubriderm** has the lowest review average score of **66.65** and a total of **82** conversations, indicating a need for improvement in customer satisfaction and engagement.

### Market Positioning
- **Neutrogena** and **Aveeno** are positioned well within the market, with Neutrogena showing strong engagement and Aveeno excelling in customer loyalty.
- **Exuviance** and **Neostrata** require strategies to enhance visibility and consumer interaction.
- **Lubriderm** needs immediate attention to improve its customer satisfaction metrics.

### Recommendations
- **Enhance Marketing Efforts**: Focus on increasing visibility and engagement for Exuviance, Neostrata, and Lubriderm through targeted marketing campaigns.
- **Leverage High NPS**: Utilize Aveeno's high NPS in promotional activities to attract new customers and reinforce brand loyalty.
- **Customer Feedback Loop**: Implement a robust feedback mechanism for Lubriderm to identify specific areas of dissatisfaction and address them promptly.

### Conclusion
Your brands are performing variably within the skincare market, with Neutrogena and Aveeno leading in engagement and loyalty, while Lubriderm requires significant improvement to enhance its market position.

Total Batch Size 10 from 50 documents


### Main Competitors in the Skin Care US Category

1. **Neutrogena**
   - **Score**: 84.61
   - **Total Conversations**: 8,234
   - **Market Position**: Leading brand with high consumer engagement and loyalty.

2. **Aquaphor**
   - **Score**: 90.28
   - **Total Conversations**: 1,043
   - **Market Position**: Strong customer satisfaction and advocacy.

3. **Garnier**
   - **Score**: 90.68
   - **Total Conversations**: 2,555
   - **Market Position**: High consumer engagement with a solid reputation.

4. **Aveeno**
   - **Score**: 88.64
   - **Total Conversations**: 1,438
   - **Market Position**: Well-regarded for customer loyalty and satisfaction.

5. **Cetaphil**
   - **Score**: 71.07
   - **Total Conversations**: 34,738
   - **Market Position**: High engagement, leading in total conversations, indicating strong market presence.

6. **Dermalogica**
   - **Score**: 79.13
   - **Total Conversations**: 1,078
   - **Market Position**: High customer affinity and satisfaction.

7. **Cerave**
   - **Score**: 69.51
   - **Total Conversations**: 8,631
   - **Market Position**: Competitive presence with moderate consumer engagement.

8. **Dove**
   - **Score**: 89.25
   - **Total Conversations**: 1,239
   - **Market Position**: Strong brand loyalty and consumer trust.

9. **La Roche-Posay**
   - **Score**: 83.01
   - **Total Conversations**: 1,688
   - **Market Position**: Recognized for quality and consumer satisfaction.

10. **Hero Cosmetics**
    - **Score**: 84.80
    - **Total Conversations**: 1,243
    - **Market Position**: Emerging brand with strong consumer loyalty.

### Conclusion
The competitive landscape is characterized by a mix of established brands like Neutrogena, Aquaphor, and Garnier, which exhibit high scores and significant consumer engagement. Brands such as Aveeno and Cetaphil also maintain strong market positions, indicating a diverse competitive environment in the Skin Care US category.

### Channel Performance Analysis

#### Overview
The performance of various channels is critical for understanding consumer engagement and optimizing marketing strategies. The following table summarizes the scores of different channels based on their effectiveness.

| Channel  | Score   |
|----------|---------|
| Social   | 70.02   |
| Ratings  | 68.52   |
| Mixed    | 59.34   |
| Reviews  | 51.47   |
| Search   | 22.13   |

#### Insights
- **Top Performer**: The **Social** channel leads with a score of **70.02**, indicating strong engagement and effectiveness in reaching consumers.
- **Second Place**: The **Ratings** channel follows closely with a score of **68.52**, suggesting that consumer reviews and ratings significantly influence brand perception and advocacy.
- **Moderate Performance**: The **Mixed** channel scores **59.34**, reflecting a moderate level of effectiveness, while the **Reviews** channel shows lower engagement at **51.47**.
- **Underperformer**: The **Search** channel has the lowest score at **22.13**, indicating a need for improvement in search engine optimization and visibility.

#### Conclusion
The analysis reveals that the **Social** and **Ratings** channels are performing well, making them key areas for marketing focus. Strategies should be developed to enhance the effectiveness of the **Mixed**, **Reviews**, and **Search** channels to improve overall consumer engagement.

In [71]:
display(Markdown(res['prompt_builder']['prompt'][0].content))

Given the conversation history and the provided supporting documents, give a brief answer to the question.
Note that supporting documents are not part of the conversation. If question can't be answered from supporting documents, say so & if needed ask clarification questions.

Answer Strictly only for the asked original question in format of Enterprise Grade Business Case Study ( dont't mention).
Include focused analysis for all possible nuances in the asked question.
If the answer is not contained within the context, reply with clarification questions. Don't assume random data.

    Conversation history:
    
        list my brands
    
        - **Brands Identified**:
  1. **Aveeno**
     - Score: 56.63
     - Total Conversations: 9820
  2. **Exuviance**
     - Score: 48.90
     - Total Conversations: 324
  3. **Lubriderm**
     - Score: 53.63
     - Total Conversations: 85
  4. **Neostrata**
     - Score: 69.35
     - Total Conversations: 174
  5. **Neutrogena**
     - Score: 74.71
     - Total Conversations: 9703

- **Key Insights**:
  - **Aveeno** leads in consumer engagement with **9820** conversations, indicating a strong market presence.
  - **Neutrogena** follows closely with **9703** conversations, reflecting significant brand interaction.
  - **Exuviance** and **Lubriderm** show lower engagement levels with **324** and **85** conversations, respectively, suggesting potential areas for improvement in brand visibility.
  - **Neostrata** has a moderate score of **69.35** with **174** conversations, indicating a niche but engaged consumer base.
    
        which is best among it
    
        - **Top Brands Identified**:
  1. **Neutrogena**: 
     - Score: 74.71 
     - Total Conversations: 9703
  2. **Aveeno**: 
     - Score: 56.63 
     - Total Conversations: 9820
  3. **Neostrata**: 
     - Score: 69.35 
     - Total Conversations: 174
  4. **Exuviance**: 
     - Score: 48.90 
     - Total Conversations: 324
  5. **Lubriderm**: 
     - Score: 53.63 
     - Total Conversations: 85

- **Analysis**:
  - **Neutrogena** stands out as the best brand among the listed options with the highest score of **74.71** and a substantial number of conversations (**9703**), indicating strong consumer engagement and brand loyalty.
  - **Aveeno** follows closely with a score of **56.63** and the highest total conversations (**9820**), suggesting good market presence but lower consumer sentiment compared to Neutrogena.
  - **Neostrata** has a moderate score of **69.35** but significantly fewer conversations (**174**), indicating a niche market presence.
  - **Exuviance** and **Lubriderm** show lower scores (**48.90** and **53.63**, respectively) and minimal conversations, suggesting they may need to enhance their marketing strategies to improve visibility and consumer engagement.

- **Conclusion**:
  - **Neutrogena** is the best-performing brand among the listed options, reflecting both high consumer engagement and positive brand perception.
    
        what makes it best
    

    Supporting documents:
    
        - **Brand Performance Metrics**:
  - **Top Scoring Brands**:
    - **Aveeno**: Score of **56.15** (Review NPS) with **1373** total conversations.
    - **Garnier**: Score of **54.98** (Review NPS) with **722** total conversations.
    - **Neutrogena**: Score of **52.36** (Buzz Share Scaled) with **29370** total conversations.
  
- **Consumer Advocacy**:
  - High scores in **Review NPS** indicate strong consumer advocacy:
    - **Aveeno** and **Garnier** show significant consumer recommendation potential.
    - **Neutrogena** leads in social engagement with a score of **97.61** (Buzz Share Scaled).

- **Engagement Levels**:
  - **Neutrogena** has the highest total conversations (**29370**), indicating robust consumer engagement and brand visibility.
  - **Cetaphil** and **The Ordinary** also show high engagement with **4114** and **3351** total conversations respectively.

- **Market Positioning**:
  - Brands like **Aveeno** and **Neutrogena** are positioned as leaders in the skincare market, leveraging both high advocacy scores and significant consumer interactions.
  - **Garnier** and **L'Oreal** maintain competitive scores, indicating strong market presence.

- **Consumer Trust and Quality Perception**:
  - High scores in **Review Average**:
    - **Cetaphil**: **77.49** with **3888** conversations.
    - **L'Oreal**: **75.91** with **1437** conversations.
  - These scores reflect consumer trust and perceived quality, essential for brand loyalty.

- **Strategic Insights**:
  - Brands with high advocacy scores and significant conversation volumes are likely to maintain or grow market share.
  - Continuous monitoring of consumer sentiment through reviews and social media engagement is crucial for adapting marketing strategies.

- **Conclusion**:
  - The combination of high scores in advocacy, significant consumer engagement, and positive trust metrics positions brands like **Aveeno**, **Garnier**, and **Neutrogena** as leaders in the skincare market, making them the best in their category.
    
        - **Brand Performance Metrics**:
  - **Cetaphil**: 
    - Score: **31.89** (Search Channel)
    - Total Conversations: **16,103**
  - **The Ordinary**: 
    - Score: **42.63** (Social Channel)
    - Total Conversations: **13,378**
  - **La Roche-Posay**: 
    - Score: **87.56** (Search Channel)
    - Total Conversations: **13,668**
  - **Cerave**: 
    - Score: **100.00** (Social Channel)
    - Total Conversations: **37,436**

- **Comparative Analysis**:
  - **Highest Score**: Cerave with a score of **100.00**, indicating exceptional brand presence and consumer engagement on social platforms.
  - **Strong Engagement**: La Roche-Posay and The Ordinary also show high scores with significant conversation volumes, suggesting effective brand strategies in engaging consumers.

- **Channel Effectiveness**:
  - **Social Channel Dominance**: Brands like Cerave and The Ordinary excel in social engagement, reflecting a strong connection with consumers and effective marketing strategies.
  - **Search Channel Performance**: La Roche-Posay and Cetaphil demonstrate high visibility in search, indicating strong brand recognition and consumer interest.

- **Consumer Sentiment**:
  - **High Total Conversations**: Brands with higher total conversations (e.g., Cerave and Cetaphil) suggest a robust consumer interest and engagement, which is critical for brand loyalty and advocacy.

- **Strategic Insights**:
  - **Focus on Social Engagement**: Brands should continue to leverage social media platforms to enhance consumer interaction and brand loyalty.
  - **Optimize Search Visibility**: Maintaining a strong presence in search results is essential for brands like Cetaphil and La Roche-Posay to capture consumer interest effectively.

- **Conclusion**: 
  - The combination of high scores, significant conversation volumes, and effective channel strategies positions brands like Cerave, La Roche-Posay, and The Ordinary as leaders in the skincare market, showcasing what makes them the best in their category.
    
        - **Brand Performance Overview**:
  - The dataset reflects various brands in the Skin Care US category, focusing on metrics such as Advocacy, Buzz, and Action.
  - Key brands include Neutrogena, L'Oreal, and Clean And Clear, which show strong performance across multiple metrics.

- **Top Brands by Advocacy Score**:
  - **Neutrogena**: 
    - Score: 75.475
    - Total Conversations: 11,489
  - **L'Oreal**: 
    - Score: 75.914
    - Total Conversations: 1,838
  - **Clean And Clear**: 
    - Score: 75.014
    - Total Conversations: 158

- **Consumer Engagement**:
  - Neutrogena leads with the highest total conversations, indicating strong consumer engagement and brand loyalty.
  - Brands like La Roche-Posay (Score: 73.232, Total Conversations: 7,587) and Drunk Elephant (Score: 73.233, Total Conversations: 1,772) also show significant engagement.

- **Buzz and Visibility**:
  - Neutrogena also excels in Buzz metrics, with a Share_Scaled score of 31.866 in June 2024, indicating high visibility and conversation around the brand.
  - Other notable brands in Buzz include Cerave (Score: 100.0) and Cetaphil (Score: 34.903), showcasing their strong presence in social discussions.

- **Trust and Advocacy**:
  - Trust scores are critical for brand perception. Brands like Olay (Score: 84.561) and Paula's Choice (Score: 85.477) demonstrate high trust levels, which correlate with consumer advocacy.
  - Advocacy scores are essential for understanding consumer willingness to recommend brands, with Clean And Clear achieving a notable score of 87.418 in December 2023.

- **Overall Brand Pulse**:
  - The Pulse Score, which combines various components, is crucial for assessing overall brand health. Brands with high Pulse Scores are likely to maintain competitive advantages.
  - Neutrogena and L'Oreal consistently rank high across multiple metrics, indicating robust brand equity and consumer loyalty.

- **Conclusion**:
  - The best-performing brands in the dataset are characterized by high scores in Advocacy, Buzz, and Trust, coupled with significant consumer engagement.
  - Continuous monitoring of these metrics can help brands adapt strategies to enhance consumer satisfaction and loyalty.
    
        - **Brand Performance Metrics**:
  - **Aveeno**: Highest NPS score of **90.76** with **1452** total conversations, indicating strong customer loyalty and advocacy.
  - **Aquaphor**: NPS score of **89.29** from **981** conversations, showcasing high customer satisfaction.
  - **Neutrogena**: NPS score of **85.63** with **6843** conversations, reflecting a solid reputation in the market.

- **Comparative Analysis**:
  - **Olay**: NPS score of **84.87** from **2712** conversations, indicating competitive performance but lower than Aveeno and Aquaphor.
  - **Hawaiian Tropic**: NPS score of **84.67** with **262** conversations, suggesting a niche but positive customer perception.
  - **Cerave**: NPS score of **86.37** with **6439** conversations, indicating a strong presence and customer satisfaction.

- **Review Metrics**:
  - **Dove**: Review average of **78.13** from **4025** conversations, indicating a generally positive reception.
  - **Neutrogena**: Review average of **74.30** from **9743** conversations, showing good customer feedback but room for improvement.
  - **Aquaphor**: Review average of **73.21** from **981** conversations, indicating solid performance.

- **Engagement Levels**:
  - **Neutrogena** leads with **6843** conversations in NPS, suggesting effective engagement strategies.
  - **Olay** and **Cerave** also show high engagement with **2712** and **6439** conversations respectively.

- **Market Positioning**:
  - Brands like **Aveeno** and **Aquaphor** are positioned as leaders in customer advocacy, which is critical for long-term brand loyalty.
  - The presence of high conversation counts alongside strong NPS scores indicates effective brand strategies in maintaining customer relationships.

- **Key Takeaways**:
  - High NPS scores correlate with strong customer advocacy and satisfaction.
  - Engagement levels (total conversations) are crucial for understanding brand performance and market presence.
  - Continuous monitoring of these metrics can help brands adapt strategies to enhance customer loyalty and market competitiveness.
    
        - **Brand Performance Overview**:
  - The dataset reflects various brands in the Skin Care US category, evaluated across multiple metrics such as Ratings, NPS (Net Promoter Score), and Buzz.
  
- **Top Brands by Rating Average**:
  - **Olay**: Score of **88.28** from **135** conversations.
  - **Garnier**: Score of **87.07** from **306** conversations.
  - **Hero Cosmetics**: Score of **80.50** from **48** conversations.
  - **Aquaphor**: Score of **82.13** from **1263** conversations.
  
- **Top Brands by NPS**:
  - **Olay**: Score of **80.04** from **135** conversations.
  - **Murad**: Score of **80.85** from **158** conversations.
  - **Garnier**: Score of **82.37** from **306** conversations.
  - **Nivea**: Score of **67.72** from **429** conversations.

- **Engagement Metrics**:
  - **Aquaphor** has the highest total conversations (**1263**), indicating strong consumer engagement despite a lower rating compared to Olay and Garnier.
  - **Neutrogena** leads in Buzz with a score of **100.00** from **32022** conversations, showcasing its dominance in market presence.

- **Competitive Analysis**:
  - Brands like **Dermalogica** and **Murad** show high scores in Advocacy and Ratings, indicating strong customer loyalty and satisfaction.
  - **Skinceuticals** has a concerning score of **0.00** in NPS, suggesting potential issues in customer satisfaction or engagement.

- **Key Insights**:
  - High scores in Ratings and NPS correlate with higher total conversations, indicating that consumer engagement is crucial for brand perception.
  - Brands with a strong Buzz score, like Neutrogena, are likely to maintain a competitive edge due to increased visibility and consumer interest.

- **Recommendations**:
  - Focus on enhancing customer engagement strategies for brands with lower scores to improve overall brand perception.
  - Leverage high-performing brands' strategies in Ratings and NPS to boost underperforming brands.
  - Monitor and address any negative sentiment or low engagement metrics, particularly for brands like Skinceuticals, to prevent further decline in customer loyalty.
    
        - **Brand Performance Metrics**:
  - **Neutrogena**: 
    - Beat Score: 75.233
    - Total Conversations: 10,110
    - Score Type: Beat Score
  - **L'Oreal**: 
    - Beat Score: 76.79
    - Total Conversations: 5,652
  - **Cerave**: 
    - Beat Score: 73.595
    - Total Conversations: 6,948
  - **Clean And Clear**: 
    - Beat Score: 75.546
    - Total Conversations: 135

- **Consumer Engagement**:
  - Neutrogena leads with the highest total conversations (10,110), indicating strong consumer engagement and brand visibility.
  - L'Oreal and Cerave also show significant engagement with 5,652 and 6,948 conversations respectively.

- **Affinity and Trust**:
  - High Beat Scores (above 75) for brands like Neutrogena, L'Oreal, and Clean And Clear suggest strong consumer affinity and trust.
  - Neutrogena's score of 75.233 reflects its effectiveness in meeting customer expectations.

- **Market Positioning**:
  - Brands with higher Beat Scores are likely perceived as superior in quality and customer satisfaction.
  - The presence of multiple brands with scores above 70 indicates a competitive market where brands are effectively resonating with consumers.

- **Strategic Insights**:
  - Brands should focus on maintaining high engagement levels to sustain or improve their Beat Scores.
  - Continuous monitoring of consumer conversations can provide insights into brand perception and areas for improvement.

- **Conclusion**:
  - The combination of high Beat Scores and significant total conversations positions brands like Neutrogena, L'Oreal, and Cerave as leaders in the skincare market, reflecting their effectiveness in building consumer loyalty and trust.
    
        - **Brand Performance Overview**:
  - The dataset reflects various brands in the Skin Care US category, evaluated on multiple metrics, particularly focusing on the **Review Net Promoter Score (NPS)**, which indicates customer loyalty and likelihood to recommend.

- **Top Performing Brands**:
  - **Lubriderm**: 
    - Score: **52.62**
    - Total Conversations: **83**
  - **Vaseline**: 
    - Score: **53.33**
    - Total Conversations: **597**
  - **Aveeno**: 
    - Score: **51.27**
    - Total Conversations: **968**
  - **Eucerin**: 
    - Score: **50.43**
    - Total Conversations: **416**
  
- **Key Insights**:
  - **High Scores**: Brands like Vaseline and Lubriderm exhibit high NPS scores, indicating strong customer advocacy and satisfaction.
  - **Engagement Levels**: Brands with higher total conversations (e.g., Vaseline and Aveeno) suggest a robust engagement strategy, leading to better customer feedback and loyalty.
  - **Competitive Edge**: Brands like Eucerin and Aveeno, with substantial conversation volumes and high scores, indicate effective marketing and product quality that resonate well with consumers.

- **Comparative Analysis**:
  - **Exuviance**: 
    - Score: **48.36**
    - Total Conversations: **4** (Low engagement despite a decent score)
  - **The Ordinary**: 
    - Score: **43.93**
    - Total Conversations: **776** (Moderate score but high engagement)
  - **Drunk Elephant**: 
    - Score: **47.77**
    - Total Conversations: **119** (Good score but lower engagement)

- **Strategic Recommendations**:
  - **Enhance Engagement**: Brands with high scores but low conversation counts (e.g., Exuviance) should focus on increasing customer interactions to leverage their positive reputation.
  - **Leverage High Advocacy**: Brands like Vaseline and Lubriderm should capitalize on their strong NPS by encouraging user-generated content and testimonials to further enhance brand visibility.
  - **Monitor Competitors**: Continuous tracking of competitors' scores and conversations can provide insights into market trends and consumer preferences, allowing for timely adjustments in strategy.

- **Conclusion**:
  - The combination of high NPS scores and substantial conversation volumes is critical for brand success in the skincare market. Brands that effectively engage with their customers while maintaining high satisfaction levels are positioned as market leaders.
    
        - **Brand Performance Overview**:
  - The dataset reflects various brands in the **Skin Care US** category, focusing on metrics related to **Advocacy**, **Buzz**, and **Trust**.
  - Key brands include **Neutrogena**, **Truskin**, **Drunk Elephant**, and **Cetaphil**, each evaluated on multiple performance metrics.

- **Top Performing Brands**:
  - **Truskin**:
    - **Social Average Score**: 89.13
    - **Total Conversations**: 116
  - **Hero Cosmetics**:
    - **Social NPS Score**: 87.585
    - **Total Conversations**: 825
  - **Neutrogena**:
    - **Social Average Score**: 84.129
    - **Total Conversations**: 6601
  - **Cetaphil**:
    - **Social NPS Score**: 70.803
    - **Total Conversations**: 1744

- **Key Metrics Analysis**:
  - **Advocacy Scores**:
    - High scores indicate strong consumer recommendation potential, with **Truskin** leading at 89.13.
    - **Neutrogena** and **Drunk Elephant** show competitive scores of 84.129 and 83.465, respectively.
  - **Engagement Levels**:
    - **Neutrogena** has the highest engagement with **6601 total conversations**, suggesting a robust consumer interaction and brand presence.
    - **Truskin**, despite fewer conversations, maintains a high score, indicating effective advocacy despite lower engagement.

- **Consumer Sentiment**:
  - Brands like **Hero Cosmetics** and **Truskin** exhibit high consumer sentiment, as reflected in their NPS and average scores.
  - Lower scores, such as **Lubriderm** (63.176) and **Vaseline** (63.954), indicate areas for improvement in consumer perception and advocacy.

- **Strategic Insights**:
  - **Focus on Advocacy**: Brands with high advocacy scores should leverage this strength in marketing strategies to enhance brand loyalty.
  - **Engagement Strategies**: Brands like **Neutrogena** should continue to engage consumers actively, as high conversation volumes correlate with better advocacy scores.
  - **Addressing Weaknesses**: Brands with lower scores should investigate consumer feedback to identify areas for improvement, particularly in product quality or customer service.

- **Conclusion**:
  - The best-performing brands are characterized by high advocacy scores and significant consumer engagement, indicating effective brand strategies and positive consumer sentiment. Brands should continue to focus on enhancing these areas to maintain and improve their market position.
    
        - **Brand Performance Overview**:
  - **Top Brands by Score**:
    - **Cetaphil**: Score of **33.31** (Social Channel) with **18,130** total conversations.
    - **Neutrogena**: Score of **100.00** (Ratings Channel) with **34,513** total conversations.
    - **La Roche-Posay**: Score of **20.92** (Ratings Channel) with **10,581** total conversations.
  
- **Engagement Metrics**:
  - **Total Conversations**: High engagement levels are observed with brands like Neutrogena and Cetaphil, indicating strong consumer interest and interaction.
  - **Channel Performance**:
    - Social channels show significant engagement for brands like Cetaphil and Neutrogena, suggesting effective social media strategies.
    - Ratings channels also reflect high scores, indicating consumer satisfaction and trust.

- **Competitive Analysis**:
  - **Neutrogena** stands out with a perfect score of **100.00**, indicating exceptional brand loyalty and consumer advocacy.
  - **La Roche-Posay** and **Dermalogica** also show strong performance, with scores of **20.92** and **11.35** respectively, highlighting their competitive positioning in the market.

- **Consumer Sentiment**:
  - High scores in the Buzz component for brands like Cetaphil and Neutrogena suggest positive consumer sentiment and brand reputation.
  - Brands with lower scores, such as **Neostrata** (1.84 in Search), may need to enhance their marketing strategies to improve visibility and consumer engagement.

- **Strategic Recommendations**:
  - **Leverage Social Media**: Brands like Cetaphil and Neutrogena should continue to capitalize on their strong social media presence to maintain and grow consumer engagement.
  - **Focus on Ratings and Reviews**: Brands should encourage more consumer reviews to boost their ratings scores, as seen with Neutrogena's high performance in this area.
  - **Address Low-Performing Brands**: Brands like Neostrata should analyze consumer feedback to identify areas for improvement and enhance their market presence.

- **Conclusion**:
  - The combination of high scores, significant total conversations, and positive consumer sentiment positions brands like Neutrogena and Cetaphil as leaders in the skincare market, while others may need to refine their strategies to enhance brand equity and consumer loyalty.
    
        - **Brand Performance Metrics**:
  - **Neutrogena**: 
    - Highest score in **Search** channel with a score of **33.57** and **Total Conversations** of **36,257**.
    - In **Ratings**, it achieved a perfect score of **100.00** with the same conversation volume, indicating strong customer satisfaction and brand loyalty.
  - **Cetaphil**: 
    - Strong performance in **Social** with a score of **33.40** and **Total Conversations** of **16,103**.
    - In **Beat Score**, it scored **35.57**, showcasing effective brand engagement.
  - **Aquaphor**: 
    - Notable in **Social** with a score of **45.80** and **Total Conversations** of **12,262**, indicating high visibility and consumer interest.

- **Competitive Standing**:
  - **La Roche-Posay**: 
    - High score of **43.03** in **Social**, with **13,668** conversations, reflecting strong brand presence.
  - **L'Oreal**: 
    - Achieved a score of **23.42** in **Social** and **76.92** in **Ratings**, indicating a balanced performance across channels.

- **Consumer Engagement**:
  - **Total Conversations** across brands indicate engagement levels:
    - Neutrogena leads with **36,257** conversations, followed by **Cetaphil** with **16,103** and **Aquaphor** with **12,262**.
  - High conversation counts correlate with higher scores, suggesting that brands with more consumer interactions tend to perform better.

- **Brand Attributes**:
  - **Buzz**: 
    - Brands like **Neutrogena** and **Cetaphil** show strong buzz metrics, indicating effective marketing strategies and consumer interest.
  - **Trust and Affinity**: 
    - High scores in **Trust** and **Affinity** metrics suggest that brands are successfully building emotional connections with consumers.

- **Recommendations**:
  - Focus on enhancing **Social Media Engagement**: Brands like **Neutrogena** and **Cetaphil** can leverage their strong social presence to further boost brand loyalty.
  - **Monitor Consumer Feedback**: Regularly analyze **Ratings** and **Reviews** to identify areas for improvement and capitalize on strengths.
  - **Increase Visibility**: Brands with lower conversation counts should consider targeted marketing strategies to enhance their presence and engagement in the market.
    

Instructions:
1. Respond insighfully as a "Pro Business Analyst" in well-structured and bulleted formats with facts, numbers (strictly up to 2 decimal only) and drawn inferences (TLdr).
2. Strictly Avoid mentioning phrases like "Based on the shared context/dataset", "mentioned", "shared" etc.
3. When need to provide analysis, Use quantitative values to provide factual accuracy instead of relying upon the text mentions.


Question: what makes it best

Answer:

In [75]:
display(Markdown(res['llm']['replies'][0].content))

- **Consumer Engagement**: 
  - Neutrogena leads with **36,257** total conversations, indicating robust consumer interaction and brand visibility.
  - High engagement levels correlate with positive brand perception and loyalty.

- **Performance Metrics**:
  - Neutrogena has a perfect score of **100.00** in the Ratings channel, reflecting exceptional customer satisfaction and trust.
  - The brand also scores **33.57** in the Search channel, showcasing strong visibility and recognition in consumer searches.

- **Advocacy and Trust**:
  - Neutrogena's high Net Promoter Score (NPS) of **85.63** indicates strong consumer advocacy, suggesting that customers are likely to recommend the brand to others.
  - The brand's ability to maintain high trust levels is critical for long-term customer loyalty.

- **Market Positioning**:
  - Positioned as a leader in the skincare market, Neutrogena effectively leverages its high advocacy scores and significant consumer interactions to maintain a competitive edge.
  - The combination of high scores in advocacy, substantial conversation volumes, and positive consumer sentiment solidifies its status as the best brand in the category.

- **Conclusion**:
  - Neutrogena's exceptional performance is attributed to its high consumer engagement, perfect ratings score, strong advocacy, and effective market positioning, making it the best brand among its competitors.

In [73]:
res #['prompt_builder']['prompt']

{'query_embedder': {'meta': {'model': 'text-embedding-ada-002',
   'usage': {'prompt_tokens': 4, 'total_tokens': 4}}},
 'memory_writer': {'messages_written': 1},
 'batch_generator': {'replies': ["- **Brand Performance Metrics**:\n  - **Top Scoring Brands**:\n    - **Aveeno**: Score of **56.15** (Review NPS) with **1373** total conversations.\n    - **Garnier**: Score of **54.98** (Review NPS) with **722** total conversations.\n    - **Neutrogena**: Score of **52.36** (Buzz Share Scaled) with **29370** total conversations.\n  \n- **Consumer Advocacy**:\n  - High scores in **Review NPS** indicate strong consumer advocacy:\n    - **Aveeno** and **Garnier** show significant consumer recommendation potential.\n    - **Neutrogena** leads in social engagement with a score of **97.61** (Buzz Share Scaled).\n\n- **Engagement Levels**:\n  - **Neutrogena** has the highest total conversations (**29370**), indicating robust consumer engagement and brand visibility.\n  - **Cetaphil** and **The Ordin